In [18]:
# ---------- Central registry of Pauli conjugation rules ----------
# Import Qiskit Library
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, Statevector
import matplotlib.pyplot as plt
from qiskit import __version__ as qiskit_version
import numpy as np
from typing import Callable, Dict, List
from collections import defaultdict

# Print the Qiskit version
print(f"Qiskit version: {qiskit_version}")


# ----------------------------------------------------------------------
# One term  α · P  in a Pauli expansion (Qiskit’s label order)
# ----------------------------------------------------------------------
class PauliPath:
    """
    One term α·P appearing in a Pauli-operator expansion.

    A ``PauliPath`` stores the complex coefficient (``coeff``) and the
    corresponding multi-qubit Pauli operator (``pauli``) expressed as a
    :class:`qiskit.quantum_info.Pauli` object.

    Parameters
    ----------
    coeff : complex
        Complex prefactor multiplying the Pauli operator.
    pauli : qiskit.quantum_info.Pauli
        Tensor product of single-qubit Pauli matrices in Qiskit label order
        (right-most character = qubit-0).

    Attributes
    ----------
    coeff : complex
        Complex prefactor of the Pauli term.
    pauli : qiskit.quantum_info.Pauli
        Underlying Pauli operator.

    Notes
    -----
    * In Qiskit’s little-endian convention, label ``'XYZ'`` represents the
      operator :math:`X \\otimes Y \\otimes Z` acting with *Z* on qubit-0,
      *Y* on qubit-1 and *X* on qubit-2.
    * The helper method :py:meth:`label_lsb` returns the label reversed so
      that the left-most character corresponds to qubit-0 (LSB-first
      representation).
    * The :py:meth:`__repr__` string follows the pattern ``'+0.5·IXY'`` for
      readability in debug output.

    Examples
    --------
    >>> from qiskit.quantum_info import Pauli
    >>> term = PauliPath(0.5, Pauli('IXY'))
    >>> term.coeff
    (0.5+0j)
    >>> term.label_lsb()
    'YXI'
    """
    __slots__ = ("coeff", "pauli")

    def __init__(self, coeff: complex, pauli: Pauli):
        self.coeff = coeff
        self.pauli = pauli

    def label_lsb(self) -> str:
        return self.pauli.to_label()[::-1] 

    def __repr__(self) -> str:                # e.g. "+0.5·IXY"
        # return f"{self.coeff:+g}·{self.pauli.to_label()}"
        return f"{self.coeff:+g}·{self.label_lsb()}"


class QuantumGate:
    """
    A registry for quantum gate Pauli conjugation rules.
    
    This class serves as a central registry for storing and retrieving
    Pauli conjugation rules for different quantum gates. Each rule defines
    how a Pauli operator transforms when conjugated by a specific gate.
    
    Attributes
    ----------
    _registry : Dict[str, Callable]
        Dictionary mapping gate names to their corresponding conjugation rule functions.
        
    Methods
    -------
    register(name)
        Decorator to register a new conjugation rule under the specified name.
    get(name)
        Retrieve the conjugation rule function for the specified gate name.
        
    Notes
    -----
    When a rule is registered using the `register` decorator, it is also
    attached as a static method to the class with an uppercase name followed
    by "gate" (e.g., "CXgate" for "cx").
    
    Each registered rule should be a pure function that takes a PauliPath
    and relevant qubit indices as input, and returns a list of PauliPath
    objects representing the result after conjugation.
    
    Examples
    --------
    >>> @QuantumGate.register("new_gate")
    >>> def _new_gate_rule(path, qubit):
    ...     # Implementation of the rule
    ...     return [transformed_path]
    ...
    >>> # Can be accessed in two ways:
    >>> QuantumGate.get("new_gate")(path, 0)
    >>> QuantumGate.NEW_GATEgate(path, 0)
    """


    # {"cx": some_function, "t": another_function, ...}
    _registry: Dict[str, Callable] = {}

    # -------- registration decorator --------
    @classmethod
    def register(cls, name: str):
        """Decorator: register a new rule under *name* and
        attach it as <NAME>gate attribute for convenience."""
        def wrapper(func: Callable):
            # add name of gate and actual function of gate to class variable _registry
            cls._registry[name] = func 

            # it gives the cls a method named "CXgate" from "_cx_rule"
            setattr(cls, f"{name.upper()}gate", staticmethod(func)) 
            return staticmethod(func)
        return wrapper

    # -------- lookup helper --------
    @classmethod
    def get(cls, name: str) -> Callable:
        '''
        Search for if _registry contains a function associated with "name"
        If can't find it, raise an error
        '''
        try:
            return cls._registry[name]
        except KeyError as exc:
            raise NotImplementedError(f"No rule registered for gate '{name}'") from exc


# ---------- Implementation of CX and T gates ----------
# !!! 需要给这玩意写test
@QuantumGate.register("cx")
def _cx_rule(path: PauliPath, ctrl: int, tgt: int) -> List[PauliPath]:
    """
    Conjugate a Pauli operator by a controlled-X (CNOT) gate.
    
    This function implements the transformation rule for conjugating a Pauli
    operator by a CNOT gate: U†PU where U is the CNOT gate from control to target.
    
    The transformation follows these rules:
    - X on control: X_c -> X_c X_t
    - Z on control: Z_c -> Z_c
    - Y on control: Y_c -> Y_c X_t
    - X on target: X_t -> X_t
    - Z on target: Z_t -> Z_c Z_t
    - Y on target: Y_t -> Z_c Y_t
    
    Parameters
    ----------
    path : PauliPath
        The Pauli path to be transformed.
    ctrl : int
        Index of the control qubit.
    tgt : int
        Index of the target qubit.
    
    Returns
    -------
    List[PauliPath]
        A list containing a single PauliPath representing the transformed operator.
        
    Notes
    -----
    The implementation uses the binary representation of Pauli operators:
    - (z=0, x=0) : I (identity)
    - (z=0, x=1) : X
    - (z=1, x=0) : Z
    - (z=1, x=1) : Y = i*X*Z
    
    The XOR operation (^=) is used to toggle bits according to the transformation rules.
    """
    z, x = path.pauli.z.copy(), path.pauli.x.copy()

    # control qubit transforms
    if x[ctrl] and not z[ctrl]:        # Xc → Xc Xt
        # x[ctrl] = 1, z[ctrl] = 0, (z[ctrl], x[ctrl]) = (0,1), is X
        # X[tgt]
        x[tgt] ^= True  # ^= is XOR, 
    elif x[ctrl] and z[ctrl]:          # Yc → Yc Xt
        x[tgt] ^= True
    # Zc unchanged

    # target qubit transforms
    if x[tgt] and z[tgt]:              # Yt → Zc Yt
        z[ctrl] ^= True
    elif z[tgt] and not x[tgt]:        # Zt → Zc Zt
        z[ctrl] ^= True
    # Xt unchanged

    return [PauliPath(path.coeff, Pauli((z, x)))]


@QuantumGate.register("t")
def _t_rule(path: PauliPath, q: int) -> List[PauliPath]:
    """Conjugate a Pauli operator by a T gate (π/4 phase on |1⟩).
    
    This function implements the transformation rules for Pauli operators
    when conjugated by a T gate (T† P T). The T gate applies a π/4 phase
    to the |1⟩ state.
    
    The transformation rules are:
    - Z → Z (unchanged)
    - X → (X - Y)/√2
    - Y → (X + Y)/√2
    - I → I (unchanged)
    
    Parameters
    ----------
    path : PauliPath
        The Pauli path to be transformed.
    q : int
        Index of the qubit on which the T gate acts.
        
    Returns
    -------
    List[PauliPath]
        A list of PauliPath objects representing the transformed operator.
        For Z and I operators, returns a single-element list with the original path.
        For X and Y operators, returns a two-element list with the transformed paths.
        
    Notes
    -----
    The T gate is a single-qubit phase gate that applies a π/4 phase shift.
    It is represented by the matrix:
    T = [[1, 0], [0, exp(iπ/4)]]
    
    This implementation uses the binary representation of Pauli operators where
    each operator is encoded by two binary arrays (z, x).
    """
    z, x = path.pauli.z.copy(), path.pauli.x.copy()

    # Z  → unchanged
    if z[q] and not x[q]:
        return [path]

    # X  → (X − Y)/√2
    if x[q] and not z[q]:
        p1 = PauliPath(path.coeff / np.sqrt(2), path.pauli)            # +X
        z2 = z.copy(); z2[q] = True                                    # +Y
        p2 = PauliPath(-path.coeff / np.sqrt(2), Pauli((z2, x)))       # −Y
        return [p1, p2]

    # Y  → (X + Y)/√2
    if x[q] and z[q]:
        p1 = PauliPath(path.coeff / np.sqrt(2), path.pauli)            # +Y
        z2 = z.copy(); z2[q] = False                                   # +X
        p2 = PauliPath(path.coeff / np.sqrt(2), Pauli((z2, x)))        # +X
        return [p1, p2]

    # I on that qubit
    return [path]


Qiskit version: 2.0.0


In [19]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli

# 你的 3-层电路:
qc = QuantumCircuit(3, name="demo")
qc.cx(0, 1)          # layer-1
for q in range(3):   # layer-2
    qc.t(q)
qc.cx(1, 2)          # layer-3
qc.draw('text')


┌───┐     
q_0: ──■──┤ T ├─────
     ┌─┴─┐├───┤     
q_1: ┤ X ├┤ T ├──■──
     ├───┤└───┘┌─┴─┐
q_2: ┤ T ├─────┤ X ├
     └───┘     └───┘

In [20]:
q2i = {}
for i, q in enumerate(qc.qubits):
    q2i[q] = i
q2i

{<Qubit register=(3, "q"), index=0>: 0,
 <Qubit register=(3, "q"), index=1>: 1,
 <Qubit register=(3, "q"), index=2>: 2}

In [21]:
for d in qc.data:
    print(d)


CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(<Qubit register=(3, "q"), index=0>, <Qubit register=(3, "q"), index=1>), clbits=())
CircuitInstruction(operation=Instruction(name='t', num_qubits=1, num_clbits=0, params=[]), qubits=(<Qubit register=(3, "q"), index=0>,), clbits=())
CircuitInstruction(operation=Instruction(name='t', num_qubits=1, num_clbits=0, params=[]), qubits=(<Qubit register=(3, "q"), index=1>,), clbits=())
CircuitInstruction(operation=Instruction(name='t', num_qubits=1, num_clbits=0, params=[]), qubits=(<Qubit register=(3, "q"), index=2>,), clbits=())
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(<Qubit register=(3, "q"), index=1>, <Qubit register=(3, "q"), index=2>), clbits=())


In [7]:
# ─── 0. 准备：初始可观测量 ───
paths = [PauliPath(1.0, Pauli("IXI"))]   #  q2 q1 q0 = I X I
print("step 0  :", paths)                # [+1·IXI]



step 0  : [+1·IXI]


In [9]:
# ─── 1. 第一层（最右边）门：CX(control=1, target=2) ───
instr = qc.data[-1]                      # 最后一条指令
rule  = QuantumGate.get(instr.operation.name)
qidx  = [qc.qubits.index(q) for q in instr.qubits]   # [1, 2]

next_paths = []
for p in paths:
    next_paths.extend(rule(p, *qidx))    # 调用 _cx_rule
next_paths

[+1·IXX]

In [22]:
qidx

[1, 2]

In [14]:
instr.qubits

(<Qubit register=(3, "q"), index=1>, <Qubit register=(3, "q"), index=2>)

In [10]:
#   1-a  可选重量截断 (这里只有演示; 设 max_weight=None 就什么也不做)
max_weight = None
if max_weight is not None:
    next_paths = [p for p in next_paths
                  if (p.pauli.x | p.pauli.z).sum() <= max_weight]
    
next_paths

[+1·IXX]

In [12]:
#   1-b  合并同标签项
bucket = defaultdict(complex)
for p in next_paths:
    bucket[p.pauli.to_label()] += p.coeff
paths = [PauliPath(c, Pauli(lbl)) for lbl, c in bucket.items()]
paths

[+1+0j·IXX]

In [13]:
#   1-c  排序（权重升序，同权重时把正实系数的排前面）
paths.sort(key=lambda p: ((p.pauli.x | p.pauli.z).sum(),
                          p.coeff.real < 0))

print("step 1  after CX(1→2):", paths)   # [+1·XXI]

step 1  after CX(1→2): [+1+0j·IXX]


In [ ]:
# ── 2. 结果：最左端的 Pauli 展开 ────────────────────────────
final_paths = paths          # per_layer[-1] 也一样
print("\nfinal_paths :", final_paths)
# 预期打印：
#   [+0.707107·IXI  -0.707107·ZYI]

# ── 3. 单量子比特期望值（|+>⊗3） ────────────────────────────
plus_ev = {'I': 1.0, 'X': 1.0, 'Y': 0.0, 'Z': 0.0}

# ── 4. 手工累加 ⟨O⟩ ─────────────────────────────────────
exp_val = 0.0
for p in final_paths:
    term = p.coeff
    print(f"\nPath {p} :")
    for letter in p.label_lsb():            # 顺序 q0, q1, q2
        print(f"  × ⟨{letter}⟩", end="")
        term *= plus_ev.get(letter, 0.0)
        if term == 0.0:
            print(" = 0  (提前终止)")
            break
        else:
            print(f" = {term}")
    exp_val += term

print(f"\nExpectation  ⟨XXI⟩_|+>⊗3  with k=2  =  {exp_val}")

In [ ]:
# ------------------------
# 试验参数
# ------------------------


TRIALS   = 500          # 运行次数
TOLERANCE = 1e-12      # np.allclose 容差

failures = 0

_single = {"I": np.eye(2, dtype=complex),
           "X": np.array([[0, 1], [1, 0]], dtype=complex),
           "Y": np.array([[0, -1j], [1j, 0]], dtype=complex),
           "Z": np.array([[1, 0], [0, -1]], dtype=complex)}
    
def pauli_to_matrix(label: str) -> np.ndarray:
    """2-qubit label (big-endian) → 4×4 matrix."""
    return np.kron(_single[label[0]], _single[label[1]])

def series_to_matrix(series):
    """list[PauliTerm] → summed 4×4 matrix."""
    acc = np.zeros((4, 4), dtype=complex)
    for term in series:
        acc += term.coeff * pauli_to_matrix(term.pauli.to_label())
    return acc
LABELS_2Q = ["".join(p) for p in itertools.product("IXYZ", repeat=2)]
for _ in range(TRIALS):
    # --- 1. 随机 SU(4) 门 -------------------
    U = random_su4()
    gate = UnitaryGate(U, label="randSU4"); gate._name = "su4"
    
    # --- 2. Very small circuit --------------
    qc = QuantumCircuit(2)
    qc.append(gate, [0, 1])   # [高位, 低位] → big-endian
    
    # --- 3. 随机输入 Pauli -------------------
    label = np.random.choice(LABELS_2Q)
    P_in  = Pauli(label)
    
    # --- 4. Propagate ------------------------
    prop   = PauliPropagator(qc)
    series = prop.propagate(PauliTerm(1.0, P_in))[-1]   # list[PauliTerm]
    
    # --- 5. 矩阵对比 -------------------------
    lhs = U.conj().T @ pauli_to_matrix(label) @ U   # U† P U
    rhs = series_to_matrix(series)                  # Σ cᵢ Pᵢ
    
    if not np.allclose(lhs, rhs, atol=TOLERANCE):
        failures += 1

print(f"Total failures: {failures} / {TRIALS}")


In [ ]:


# --------------------------------------------------------------------
SYMS_STATE = "01+-rl"
SYMS_PAULI = "IXYZ"

def random_state_label(n: int) -> str:
    """Random product-state label, big-endian (left char = qubit-n-1)."""
    return "".join(random.choice(SYMS_STATE) for _ in range(n))


def random_pauli_label(n: int) -> str:
    """Random Pauli label with at least one non-I."""
    label = ["I"] * n
    idx = random.randrange(n)               # ensure one non-I
    label[idx] = random.choice("XYZ")
    for i in range(n):
        if i != idx:
            label[i] = random.choice(SYMS_PAULI)
    return "".join(label)

def build_random_circuit(n: int, gate_count: int) -> QuantumCircuit:
    """Generate a random circuit containing T, CX and SU(4) gates."""
    qc = QuantumCircuit(n, name=f"rand_mix_{n}q_{gate_count}g")
    for _ in range(gate_count):
        gtype = random.choice(("t", "cx", "su4"))
        if gtype == "t":
            qc.t(random.randrange(n))
        elif gtype == "cx":
            ctrl, tgt = random.sample(range(n), 2)
            qc.cx(ctrl, tgt)
        else:                               # SU(4)
            q1, q2 = random.sample(range(n), 2)
            gate = UnitaryGate(random_su4(), label="randSU4")
            gate._name = "su4"              # dispatch → _su4_rule
            qc.append(gate, [q1, q2])
    return qc

# --------------------------------------------------------------------
TRIALS = 100
TOL     = 1e-10

for _ in tqdm(range(TRIALS)):
    """Compare PauliPropagator vs exact state-vector for random circuits."""
    n          = random.randint(3, 10)
    gate_count = random.randint(5, 10)
    qc         = build_random_circuit(n, gate_count)

    state_lbl  = random_state_label(n)
    pauli_lbl  = random_pauli_label(n)
    observable = Pauli(pauli_lbl)

    # --- PauliPropagator expectation ---------------------------------
    prop = PauliPropagator(qc)
    layers   = prop.propagate(PauliTerm(1.0, observable))
    prop_ev  = prop.expectation_pauli_sum(layers[-1], state_lbl)

    # --- Full state-vector expectation -------------------------------
    sv_ev = Statevector.from_label(state_lbl).evolve(qc).expectation_value(
        SparsePauliOp.from_list([(pauli_lbl, 1.0)])).real

    print(abs(prop_ev - sv_ev) < TOL)